 <font size=6><b>DSC 522 Ghost Hunting: Finding a Botnet in Netflow Traffic<br>
    <font size=4>Qs.2 c and Qs.3 c,d,k done by Emma Delehanty

In [ ]:
# Please note!!!  The group had to expand the RAM available to Jupyter for this code to run. 
# We exapnded the RAM to 4GB and recommend you do the same.  Please see the answer to the below
# stackoverflow page concerning expanding the amount of max_buffer_size. 
# https://stackoverflow.com/questions/57948003/how-to-increase-jupyter-notebook-memory-limit

In [1]:
#Importing packages
import pandas as pd
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
#Reading df
df = pd.read_csv(r'capture20110815-3.pcap.netflow_v2.txt', delim_whitespace=True)


In [3]:
#Dropping unneccesary columns
df.drop(['Src','Bytes','Flows','Label','Labels','Flags'],axis=1,inplace = True)

In [4]:
#Splitting Prot and IP columns into their respective IPs and Ports
df[['a','b']] = df.Prot.str.split(":",n=1,expand=True)
df[['c','d']] = df.IP.str.split(":",n=1,expand=True)

#Drop those columns to avaoid confusion after splitting
df.drop(['Prot','IP'],axis=1,inplace = True)

In [5]:
#Renaming the columns using their proper names from the excel file
df.rename(columns = {list(df)[1]:'Flow_Start',list(df)[2]:'Duration',list(df)[3]:'Protocol',list(df)[10]:'Source_IP',list(df)[11]:'Source_Port',
                    list(df)[12]:'Destination_IP',list(df)[13]:'Destination_Port',list(df)[4]:'Flags',list(df)[5]:'Type_of_Service',
                    list(df)[6]:'Packets',list(df)[7]:'Bytes',list(df)[8]:'Labels',list(df)[9]:'Additional_Label'}, inplace = True)

In [6]:
df.drop(['Date'],axis=1,inplace = True)

In [7]:
df['Label'] = df['Labels'].astype(str) + df['Additional_Label'].astype(str)

In [8]:
#Dropping unneccesary columns
df.drop(['Labels','Additional_Label'],axis=1,inplace = True)

In [9]:
df['Label'] = df['Label'].str.replace(r'nan', '')

<font size=5><font color='red'>Qs.2.c.	Source IP / Destination IP – Do certain IPv4 octaves have an effect on botnet classification? What effect do internal/external, broadcast/other special IPs, etc. have on botnet classification?

ipaddress — IPv4/IPv6 manipulation library
https://docs.python.org/3/library/ipaddress.html

### Checking Source IP (check ip if its legal or ipv4 or ipv6)

In [10]:
import ipaddress
def checkip(ip):
    try:
        ip_class=ipaddress.ip_network(ip)
        return ip_class.__class__.__name__
    except:
        return 'invalid'

In [11]:
source1 = df[['Source_IP']]
source1

,Source_IP
0,90.177.154.197
1,147.32.84.144
2,198.36.38.132
3,41.103.64.21
4,147.32.86.183
...,...
1299085,147.32.84.2
1299086,147.32.84.165
1299087,147.32.84.171
1299088,95.211.93.1


In [12]:
unique_ips = source1['Source_IP'].unique()
unique_ips = pd.Series(unique_ips, index = unique_ips)
unique_ips.nunique()

94235

In [13]:
df['Network_class']=source1['Source_IP'].map(unique_ips.apply(checkip))
df

,Flow_Start,Duration,Protocol,Flags,Type_of_Service,Packets,Bytes,Source_IP,Source_Port,Destination_IP,Destination_Port,Label,Network_class
0,17:13:40.449,4.230,TCP,PA_,0,180,12628,90.177.154.197,20127,147.32.84.144,22,Background,IPv4Network
1,17:13:40.449,4.204,TCP,PA_,0,329,478578,147.32.84.144,22,90.177.154.197,20127,Background,IPv4Network
2,17:13:40.453,4.985,TCP,PA_,0,94,6212,198.36.38.132,55530,147.32.86.183,443,Background,IPv4Network
3,17:13:40.456,4.975,UDP,INT,0,182,61555,41.103.64.21,54617,147.32.86.183,51246,Background,IPv4Network
4,17:13:40.456,4.975,TCP,PA_,0,185,66524,147.32.86.183,443,198.36.38.132,55530,Background,IPv4Network
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1299085,19:17:33.765,0.134,TCP,SPA_,0,5,3841,147.32.84.2,80,66.194.55.249,43931,Background,IPv4Network
1299086,19:17:33.811,0.930,TCP,FPA_,0,63,85828,147.32.84.165,1386,212.117.171.138,65500,Botnet,IPv4Network
1299087,19:17:33.811,0.000,TCP,PA_,0,1,71,147.32.84.171,43352,95.211.93.1,30091,Background,IPv4Network
1299088,19:17:33.828,0.000,TCP,A_,0,1,66,95.211.93.1,30091,147.32.84.171,43352,Background,IPv4Network


In [14]:
df['Network_class'].unique()

array(['IPv4Network', 'invalid'], dtype=object)

In [15]:
print(df.groupby('Network_class').size())

Network_class
IPv4Network    1298114
invalid            976
dtype: int64


<font color='blue'><font size=3>Only about 976 invalid,filter out invalid IP,dataset has 1298114 IPv4Network

In [16]:
df=df[df.Network_class !='invalid']
df

,Flow_Start,Duration,Protocol,Flags,Type_of_Service,Packets,Bytes,Source_IP,Source_Port,Destination_IP,Destination_Port,Label,Network_class
0,17:13:40.449,4.230,TCP,PA_,0,180,12628,90.177.154.197,20127,147.32.84.144,22,Background,IPv4Network
1,17:13:40.449,4.204,TCP,PA_,0,329,478578,147.32.84.144,22,90.177.154.197,20127,Background,IPv4Network
2,17:13:40.453,4.985,TCP,PA_,0,94,6212,198.36.38.132,55530,147.32.86.183,443,Background,IPv4Network
3,17:13:40.456,4.975,UDP,INT,0,182,61555,41.103.64.21,54617,147.32.86.183,51246,Background,IPv4Network
4,17:13:40.456,4.975,TCP,PA_,0,185,66524,147.32.86.183,443,198.36.38.132,55530,Background,IPv4Network
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1299085,19:17:33.765,0.134,TCP,SPA_,0,5,3841,147.32.84.2,80,66.194.55.249,43931,Background,IPv4Network
1299086,19:17:33.811,0.930,TCP,FPA_,0,63,85828,147.32.84.165,1386,212.117.171.138,65500,Botnet,IPv4Network
1299087,19:17:33.811,0.000,TCP,PA_,0,1,71,147.32.84.171,43352,95.211.93.1,30091,Background,IPv4Network
1299088,19:17:33.828,0.000,TCP,A_,0,1,66,95.211.93.1,30091,147.32.84.171,43352,Background,IPv4Network


In [17]:
# chech IP if private
from ipaddress import IPv4Address
import ipaddress
def check_private(ip):
    try:
        ip_private=IPv4Address(ip).is_private
        return ip_private
    except:
        return 'invalid'

In [18]:
df['Network_type']=source1['Source_IP'].map(unique_ips.apply(check_private))

In [19]:
df['Network_type'].unique()

array([False, True], dtype=object)

In [20]:
print(df.groupby('Network_type').size())

Network_type
False    1295939
True        2175
dtype: int64


<font color='blue'><font size=3>2175 are private, rest are not private network 

In [21]:
from ipaddress import IPv4Address
import ipaddress
def check_global(ip):
    try:
        ip_private=IPv4Address(ip).is_global
        return ip_private
    except:
        return 'invalid'
df['Network_global']=source1['Source_IP'].map(unique_ips.apply(check_global))

In [22]:
df['Network_global'].unique()

array([True, False], dtype=object)

In [23]:
print(df.groupby('Network_global').size())

Network_global
False       2175
True     1295939
dtype: int64


<font color='blue'><font size=3>1295939 are private, rest are not private network

In [24]:
from ipaddress import IPv4Address
import ipaddress
def check_multi(ip):
    try:
        ip_private=IPv4Address(ip).is_unspecified
        return ip_private
    except:
        return 'invalid'
df['Network_is_unspecified']=source1['Source_IP'].map(unique_ips.apply(check_multi))

In [25]:
df['Network_is_unspecified'].unique()

array([False, True], dtype=object)

In [26]:
print(df.groupby('Network_is_unspecified').size())

Network_is_unspecified
False    1297547
True         567
dtype: int64


 <font color='blue'><font size=3>567 are unspecified, rest are not unspecified network

In [27]:
from ipaddress import IPv4Address
import ipaddress
def check_reserved(ip):
    try:
        ip_private=IPv4Address(ip).is_reserved
        return ip_private
    except:
        return 'invalid'
df['Network_is_reserved']=source1['Source_IP'].map(unique_ips.apply(check_reserved))

In [28]:
df['Network_is_reserved'].unique()

array([False], dtype=object)

In [29]:
print(df.groupby('Network_is_reserved').size())

Network_is_reserved
False    1298114
dtype: int64


<font color='blue'><font size=3>All IP are not reserved

### Checking Destination_IP

In [30]:
source2 = df[['Destination_IP']]
source2

,Destination_IP
0,147.32.84.144
1,90.177.154.197
2,147.32.86.183
3,147.32.86.183
4,198.36.38.132
...,...
1299085,66.194.55.249
1299086,212.117.171.138
1299087,95.211.93.1
1299088,147.32.84.171


In [31]:
import ipaddress
def checkip(ip):
    try:
        ip_class=ipaddress.ip_network(ip)
        return ip_class.__class__.__name__
    except:
        return 'invalid'
unique_ipss = source2['Destination_IP'].unique()
unique_ipss = pd.Series(unique_ipss, index = unique_ipss)
unique_ipss.nunique()

97093

In [32]:
df['D_Network_class']=source2['Destination_IP'].map(unique_ipss.apply(checkip))

In [33]:
df['D_Network_class'].unique()
print(df.groupby('D_Network_class').size())

D_Network_class
IPv4Network    1298114
dtype: int64


<font color='blue'><font size=3>All IP address in this dataset are IPv4Network

In [34]:
from ipaddress import IPv4Address
import ipaddress
def check_private(ip):
    try:
        ip_private=IPv4Address(ip).is_private
        return ip_private
    except:
        return 'invalid'
df['D_Network_type']=source2['Destination_IP'].map(unique_ipss.apply(check_private))
df['D_Network_type'].unique()

array([False,  True])

In [35]:
Only_type_false = df[df['D_Network_type']==False]
print(Only_type_false.groupby('Label').size())

Label
Background                 1213890
Botnet                       21112
BotnetFILTER_LEGITIMATE        648
LEGITIMATE                   59190
dtype: int64


In [36]:
Only_type_true = df[df['D_Network_type']==True]
print(Only_type_true.groupby('Label').size())

Label
Background    3274
dtype: int64


In [37]:
from ipaddress import IPv4Address
import ipaddress
def check_global(ip):
    try:
        ip_private=IPv4Address(ip).is_global
        return ip_private
    except:
        return 'invalid'
df['D_Network_global']=source2['Destination_IP'].map(unique_ipss.apply(check_global))
df['D_Network_global'].unique()

array([ True, False])

In [38]:
print(df.groupby('D_Network_global').size())

D_Network_global
False       3274
True     1294840
dtype: int64


In [39]:
Only_global_true = df[df['D_Network_global']==True]
print(Only_global_true.groupby('Label').size())

Label
Background                 1213890
Botnet                       21112
BotnetFILTER_LEGITIMATE        648
LEGITIMATE                   59190
dtype: int64


In [40]:
Only_global_flase = df[df['D_Network_global']==False]
print(Only_global_flase.groupby('Label').size())

Label
Background    3274
dtype: int64


In [41]:
from ipaddress import IPv4Address
import ipaddress
def check_multi(ip):
    try:
        ip_private=IPv4Address(ip).is_unspecified
        return ip_private
    except:
        return 'invalid'
df['D_Network_is_unspecified']=source2['Destination_IP'].map(unique_ipss.apply(check_multi))
df['D_Network_is_unspecified'].unique()

array([False])

In [42]:
print(df.groupby('D_Network_is_unspecified').size())

D_Network_is_unspecified
False    1298114
dtype: int64


In [43]:
from ipaddress import IPv4Address
import ipaddress
def check_reserved(ip):
    try:
        ip_private=IPv4Address(ip).is_reserved
        return ip_private
    except:
        return 'invalid'
df['D_Network_is_reserved']=source2['Destination_IP'].map(unique_ipss.apply(check_reserved))
df['D_Network_is_reserved'].unique()

array([False,  True])

In [44]:
print(df.groupby('D_Network_is_reserved').size())

D_Network_is_reserved
False    1297483
True         631
dtype: int64


### check the ip which class  - class A ,class B

In [45]:
# get class A Destination_IP
from ipaddress import IPv4Address, IPv4Network
import ipaddress
def check(unique_ipss):
    try:
        classA = IPv4Network(("10.0.0.0", "255.0.0.0"))  # or IPv4Network("10.0.0.0/8")
        unique_ipss in classA
        return 'True'
    except:
        return 'False'

In [46]:
df['D_class_A']=source2['Destination_IP'].map(unique_ipss.apply(check))
df

,Flow_Start,Duration,Protocol,Flags,Type_of_Service,Packets,Bytes,Source_IP,Source_Port,Destination_IP,...,Network_type,Network_global,Network_is_unspecified,Network_is_reserved,D_Network_class,D_Network_type,D_Network_global,D_Network_is_unspecified,D_Network_is_reserved,D_class_A
0,17:13:40.449,4.230,TCP,PA_,0,180,12628,90.177.154.197,20127,147.32.84.144,...,False,True,False,False,IPv4Network,False,True,False,False,False
1,17:13:40.449,4.204,TCP,PA_,0,329,478578,147.32.84.144,22,90.177.154.197,...,False,True,False,False,IPv4Network,False,True,False,False,False
2,17:13:40.453,4.985,TCP,PA_,0,94,6212,198.36.38.132,55530,147.32.86.183,...,False,True,False,False,IPv4Network,False,True,False,False,False
3,17:13:40.456,4.975,UDP,INT,0,182,61555,41.103.64.21,54617,147.32.86.183,...,False,True,False,False,IPv4Network,False,True,False,False,False
4,17:13:40.456,4.975,TCP,PA_,0,185,66524,147.32.86.183,443,198.36.38.132,...,False,True,False,False,IPv4Network,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1299085,19:17:33.765,0.134,TCP,SPA_,0,5,3841,147.32.84.2,80,66.194.55.249,...,False,True,False,False,IPv4Network,False,True,False,False,False
1299086,19:17:33.811,0.930,TCP,FPA_,0,63,85828,147.32.84.165,1386,212.117.171.138,...,False,True,False,False,IPv4Network,False,True,False,False,False
1299087,19:17:33.811,0.000,TCP,PA_,0,1,71,147.32.84.171,43352,95.211.93.1,...,False,True,False,False,IPv4Network,False,True,False,False,False
1299088,19:17:33.828,0.000,TCP,A_,0,1,66,95.211.93.1,30091,147.32.84.171,...,False,True,False,False,IPv4Network,False,True,False,False,False


In [47]:
print(df.groupby('D_class_A').size())

D_class_A
False    1298114
dtype: int64


In [48]:
# get class B Destination_IP
from ipaddress import IPv4Address, IPv4Network
import ipaddress
def check(unique_ipss):
    try:
        classB =IPv4Network(("172.16.0.0", "255.240.0.0"))  # or IPv4Network("172.16.0.0/12")
        IPv4Address(unique_ipss) in classB
        return 'True'
    except:
        return 'False'

In [49]:
df['D_class_B']=source2['Destination_IP'].map(unique_ipss.apply(check))

In [50]:
df['D_class_B'].unique()

array(['True'], dtype=object)

In [51]:
print(df.groupby('D_class_B').size())

D_class_B
True    1298114
dtype: int64


### Add all new features to dataframe

In [52]:
from sklearn import preprocessing
#creating labelEncoder
label_encoder = preprocessing.LabelEncoder()

In [53]:
df_features = df[['Network_type','Network_global','Network_is_unspecified','Network_is_reserved',
                  'D_Network_type','D_Network_global', 'D_Network_is_unspecified', 'D_Network_is_reserved']]

In [54]:
#check new dataset info
df_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1298114 entries, 0 to 1299089
Data columns (total 8 columns):
 #   Column                    Non-Null Count    Dtype 
---  ------                    --------------    ----- 
 0   Network_type              1298114 non-null  object
 1   Network_global            1298114 non-null  object
 2   Network_is_unspecified    1298114 non-null  object
 3   Network_is_reserved       1298114 non-null  object
 4   D_Network_type            1298114 non-null  bool  
 5   D_Network_global          1298114 non-null  bool  
 6   D_Network_is_unspecified  1298114 non-null  bool  
 7   D_Network_is_reserved     1298114 non-null  bool  
dtypes: bool(4), object(4)
memory usage: 54.5+ MB


In [55]:
# label encoding 
mymap = {'True':0,'False':1}
df_features1= df_features.applymap(lambda s: mymap.get(s) if s in mymap else s)
df_features1

,Network_type,Network_global,Network_is_unspecified,Network_is_reserved,D_Network_type,D_Network_global,D_Network_is_unspecified,D_Network_is_reserved
0,False,True,False,False,False,True,False,False
1,False,True,False,False,False,True,False,False
2,False,True,False,False,False,True,False,False
3,False,True,False,False,False,True,False,False
4,False,True,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...
1299085,False,True,False,False,False,True,False,False
1299086,False,True,False,False,False,True,False,False
1299087,False,True,False,False,False,True,False,False
1299088,False,True,False,False,False,True,False,False


In [56]:
for u in df_features1.columns:
    if df_features1[u].dtype == 'bool':
        df_features1[u] = df_features[u].astype('int')
df_features1

,Network_type,Network_global,Network_is_unspecified,Network_is_reserved,D_Network_type,D_Network_global,D_Network_is_unspecified,D_Network_is_reserved
0,0,1,0,0,0,1,0,0
1,0,1,0,0,0,1,0,0
2,0,1,0,0,0,1,0,0
3,0,1,0,0,0,1,0,0
4,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...
1299085,0,1,0,0,0,1,0,0
1299086,0,1,0,0,0,1,0,0
1299087,0,1,0,0,0,1,0,0
1299088,0,1,0,0,0,1,0,0


In [57]:
df_features2 = df_features1[['Network_global','Network_is_unspecified','Network_is_reserved',
                 'D_Network_global', 'D_Network_is_unspecified', 'D_Network_is_reserved']]
df_features2['Label']=df['Label']
df_features2

,Network_global,Network_is_unspecified,Network_is_reserved,D_Network_global,D_Network_is_unspecified,D_Network_is_reserved,Label
0,1,0,0,1,0,0,Background
1,1,0,0,1,0,0,Background
2,1,0,0,1,0,0,Background
3,1,0,0,1,0,0,Background
4,1,0,0,1,0,0,Background
...,...,...,...,...,...,...,...
1299085,1,0,0,1,0,0,Background
1299086,1,0,0,1,0,0,Botnet
1299087,1,0,0,1,0,0,Background
1299088,1,0,0,1,0,0,Background


### Balancing the dataset using down-sampling


In [58]:
# Separate majority and minority classes
majority = df_features2.loc[df_features2.Label.isin(['Background', 'LEGITIMATE'])]
minority = df_features2.loc[df_features2.Label.isin(['Botnet', 'BotnetFILTER_LEGITIMATE'])]

In [59]:
from sklearn.utils import resample
df_majority_upsampled = resample(majority, 
                                 replace=False,     # sample with replacement
                                 n_samples=21760,   # to match minority class
                                 random_state=1) # reproducible results

In [60]:
downsampled = pd.concat([minority, df_majority_upsampled])

In [61]:
downsampled['Label'].value_counts()

Botnet                     21112
Background                 20713
LEGITIMATE                  1047
BotnetFILTER_LEGITIMATE      648
Name: Label, dtype: int64

In [62]:
X=downsampled.drop('Label',axis=1)
y=downsampled[['Label']]


In [63]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

### Logistic Regression (only using these new features)

In [64]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(random_state=42,solver='lbfgs')
log_reg.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [65]:
from sklearn.model_selection import cross_val_score
cvs=cross_val_score(log_reg, X_test, y_test, cv=5, scoring='accuracy')
print(cvs)
print("Average of the scores",np.mean(cvs))

[0.48928845 0.4881317  0.48850575 0.48888889 0.48850575]
Average of the scores 0.4886641059626825


In [66]:
Target =df.iloc[:,11:12]
Target.rename(columns = {12:'label'}, inplace = True) 
Target.head()

,Label
0,Background
1,Background
2,Background
3,Background
4,Background


In [67]:
#Classifying labels into two categories
#botnet,botnetfilter_leg = Yes
#Background,legitimate = No
a = ['Botnet', 'BotnetFILTER_LEGITIMATE']
Target['yes/no'] = np.where(Target['Label'].isin(a), 'yes', 'no')

In [68]:
d = pd.merge(Target, df_features2, right_index=True,left_index=True)
d

,Label_x,yes/no,Network_global,Network_is_unspecified,Network_is_reserved,D_Network_global,D_Network_is_unspecified,D_Network_is_reserved,Label_y
0,Background,no,1,0,0,1,0,0,Background
1,Background,no,1,0,0,1,0,0,Background
2,Background,no,1,0,0,1,0,0,Background
3,Background,no,1,0,0,1,0,0,Background
4,Background,no,1,0,0,1,0,0,Background
...,...,...,...,...,...,...,...,...,...
1299085,Background,no,1,0,0,1,0,0,Background
1299086,Botnet,yes,1,0,0,1,0,0,Botnet
1299087,Background,no,1,0,0,1,0,0,Background
1299088,Background,no,1,0,0,1,0,0,Background


In [69]:
d['yes/no'].value_counts()

no     1276354
yes      21760
Name: yes/no, dtype: int64

In [70]:
majo = d.loc[d['yes/no']=='no']
mino = d.loc[d['yes/no']=='yes']

In [71]:
# downsample majority class
maj_dsampled = resample(majo, 
                        replace=False,     # sample with replacement
                        n_samples=21760,   # to match minority class
                        random_state=1) # reproducible results

In [72]:
dsampled = pd.concat([mino, maj_dsampled])
dsampled

,Label_x,yes/no,Network_global,Network_is_unspecified,Network_is_reserved,D_Network_global,D_Network_is_unspecified,D_Network_is_reserved,Label_y
4117,Botnet,yes,1,0,0,1,0,0,Botnet
6269,Botnet,yes,1,0,0,1,0,0,Botnet
8919,Botnet,yes,1,0,0,1,0,0,Botnet
10127,Botnet,yes,1,0,0,1,0,0,Botnet
11546,Botnet,yes,1,0,0,1,0,0,Botnet
...,...,...,...,...,...,...,...,...,...
996336,Background,no,1,0,0,1,0,0,Background
789958,Background,no,1,0,0,1,0,0,Background
164550,Background,no,1,0,0,1,0,0,Background
210363,Background,no,1,0,0,1,0,0,Background


In [73]:
dsampled['yes/no'].value_counts()

no     21760
yes    21760
Name: yes/no, dtype: int64

In [74]:
y=dsampled[['yes/no']]

In [75]:
X1=dsampled.drop(['yes/no','Label_y','Label_x'],axis=1)

In [76]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.3, random_state=0)

In [77]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [78]:
y_pred = model.predict(X_test) 
model.score(X_test, y_test)

0.5014552696078431

### Adding  original Flags feature to the dataset 

In [79]:
c=d.drop(['yes/no','Label_y'],axis=1)

In [80]:
Flags_Copy=df[['Flags']].copy()
import numpy as np
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
cat_data=np.array(Flags_Copy).reshape(-1, 1)
cat_data

array([['PA_'],
       ['PA_'],
       ['PA_'],
       ...,
       ['PA_'],
       ['A_'],
       ['INT']], dtype=object)

In [81]:
# hot en-coding flgas feature,concat with feature dataframe  
cat_hot=enc.fit_transform(cat_data)
cat=cat_hot.toarray()
X_cat=np.concatenate((c,cat),axis=1)
New_df = pd.DataFrame(X_cat)
New_df.rename(columns = {0:'Label'}, inplace = True) 
New_df

,Label,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
0,Background,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Background,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Background,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Background,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Background,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298109,Background,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1298110,Botnet,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1298111,Background,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1298112,Background,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
maj = New_df.loc[New_df.Label.isin(['Background', 'LEGITIMATE'])]
mino =New_df.loc[New_df.Label.isin(['Botnet', 'BotnetFILTER_LEGITIMATE'])]

In [83]:
df_majority_upsampled = resample(maj, 
                                 replace=False,     # sample with replacement
                                 n_samples=21760,   # to match minority class
                                 random_state=1) # reproducible results
 

In [84]:
# Combine majority class with downsampled majority class
downsampled = pd.concat([mino, df_majority_upsampled])
downsampled

,Label,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
4115,Botnet,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6266,Botnet,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8914,Botnet,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10121,Botnet,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11539,Botnet,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995621,Background,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
789406,Background,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
164446,Background,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
210231,Background,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
y=downsampled[['Label']]
X=downsampled.drop('Label',axis=1)

In [86]:
# do another logistc regression 
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [87]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [88]:
y_pred = model.predict(X_test)
model.score(X_test, y_test)

0.6811427696078431

<font color='blue'><font size=3>As we see adding the new features do improve the model accuracy.
    

<font size=5><font color='red'>3.	Once the final variable set is created, which of the following techniques most effectively classifies or aides in classifying botnet traffic?<br><br>
    <font size=4>c.	Naive Bayes<br><br>
        d.	Gini Decision Tree<br><br>
        k.	Hierarchical Clustering

## Naive Bayes Classifiers(with unbalancing data)

In [89]:
# read csv final_X with target 
last_X = pd.read_csv(r'final_X_with_target.csv')
last_X

,Unnamed: 0,Flow_Start,Source_IP,Destination_IP,Duration,Packets,Bytes,A_,CON,ECO,...,843,877,88,888,916,98,993,995,Label,yes/no
0,0,17:13:40.449,90.177.154.197,147.32.84.144,4.230,180,12628,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Background,no
1,1,17:13:40.449,147.32.84.144,90.177.154.197,4.204,329,478578,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Background,no
2,2,17:13:40.453,198.36.38.132,147.32.86.183,4.985,94,6212,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Background,no
3,3,17:13:40.456,41.103.64.21,147.32.86.183,4.975,182,61555,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Background,no
4,4,17:13:40.456,147.32.86.183,198.36.38.132,4.975,185,66524,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Background,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1299085,1299085,19:17:33.765,147.32.84.2,66.194.55.249,0.134,5,3841,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Background,no
1299086,1299086,19:17:33.811,147.32.84.165,212.117.171.138,0.930,63,85828,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Botnet,yes
1299087,1299087,19:17:33.811,147.32.84.171,95.211.93.1,0.000,1,71,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Background,no
1299088,1299088,19:17:33.828,95.211.93.1,147.32.84.171,0.000,1,66,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Background,no


In [90]:
y=last_X[['yes/no']]

In [91]:
# Import LabelEncoder
from sklearn import preprocessing
#creating labelEncoder
label_encoder = preprocessing.LabelEncoder()
# Encode one categorical feature
y1 = y.apply(preprocessing.LabelEncoder().fit_transform)

In [92]:
# originally dropping label yes/no','Flow_Start','Source_IP','Destination_IP','Duration','Packets','Bytes
X=last_X.drop(columns=['Label','yes/no','Flow_Start','Source_IP','Destination_IP','Duration','Packets','Bytes','Unnamed: 0'],axis=1) 
X.head()

,A_,CON,ECO,ECR,FAC_,FAU_,FA_,FPAC_,FPA_,FRAEC_,...,83,832,843,877,88,888,916,98,993,995
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [93]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size=0.3, random_state=0)

In [94]:
#Create a Gaussian Classifier
model = GaussianNB()

# Train the model using the training sets
model.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [95]:
y_pred = model.predict(X_test) 
model.score(X_test, y_test)

0.9984296699997691

## GINI Decision Tree Classifier(with unbalancing data)

In [96]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size=0.3, random_state=0)

dtree = DecisionTreeClassifier(criterion='gini')
dtree.fit(X_train,y_train)
predictions = dtree.predict(X_test)
dtree.score(X_test, y_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,predictions))

[[383147      0]
 [     4   6576]]


In [97]:
dtree.score(X_test, y_test)

0.9999897364052273

## Naive Bayes Classifiers(with balancing data)

In [98]:
#After combining labels into Botnet-yes or no
last_X['yes/no'].value_counts()

no     1277330
yes      21760
Name: yes/no, dtype: int64

In [99]:
majo = last_X.loc[last_X['yes/no']=='no']
mino = last_X.loc[last_X['yes/no']=='yes']

In [100]:
# downsample majority class
maj_dsampled = resample(majo, 
                        replace=False,     # sample with replacement
                        n_samples=21760,   # to match minority class
                        random_state=1)    # reproducible results

In [101]:
dsampled = pd.concat([mino, maj_dsampled])
dsampled['yes/no'].value_counts()

no     21760
yes    21760
Name: yes/no, dtype: int64

In [102]:
y2=dsampled[['yes/no']]
X2=dsampled.drop(['yes/no','Label','Flow_Start','Source_IP','Destination_IP','Unnamed: 0'],axis=1)

X2.fillna(0, inplace=True)
X2.head()

,Duration,Packets,Bytes,A_,CON,ECO,ECR,FAC_,FAU_,FA_,...,83,832,843,877,88,888,916,98,993,995
4117,0.000,1,263,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6269,0.000,1,66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8919,2.712,4,256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10127,0.000,1,64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11546,0.000,1,64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [103]:
from sklearn import metrics
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size=0.3, random_state=0)

#Create a Gaussian Classifier
model = GaussianNB()
# Train the model using the training sets
model.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [104]:
y_pred = model.predict(X_test)
model.score(X_test, y_test)

0.5087316176470589

## GINI Decision Tree Classifier(with balancing data)

In [105]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size=0.3, random_state=0)

dtree = DecisionTreeClassifier(criterion='gini')
dtree.fit(X_train,y_train)
predictions = dtree.predict(X_test)
dtree.score(X_test, y_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,predictions))

[[6515    7]
 [   6 6528]]


In [106]:
dtree.score(X_test, y_test)

0.9990042892156863

## Hierarchical clustering 
<font color='blue'><font size=3>Hierarchical clustering in most variants needs O(n²) memory. Because of this, most implementations will fail at around 65535 instances, when they hit the 32 bit mark (some may fail at 32k already). But just do the math: n * n * 8 bytes for double precision: how much memory would you need? so we need downsize the data since our dataset is too large.

In [107]:
# here we only using 20 % of dataset which about 8704 rows
orig=dsampled.sample(frac=0.2, replace=True, random_state=1)
orig

,Unnamed: 0,Flow_Start,Source_IP,Destination_IP,Duration,Packets,Bytes,A_,CON,ECO,...,843,877,88,888,916,98,993,995,Label,yes/no
451907,451907,17:51:28.932,99.235.201.3,147.32.84.229,0.000,1,60,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Background,no
799441,799441,18:24:09.960,147.32.84.165,147.32.80.9,0.000,1,71,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Botnet,yes
400755,400755,17:46:43.605,147.32.84.165,147.32.80.9,0.000,1,74,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Botnet,yes
858709,858709,18:30:01.988,70.119.95.44,147.32.84.229,0.000,1,73,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Background,no
533522,533522,17:59:13.875,147.32.84.165,87.248.120.148,3.004,2,124,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Botnet,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
925335,925335,18:36:24.574,62.245.66.71,147.32.85.74,0.147,3,422,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Background,no
787396,787396,18:22:52.425,147.32.84.165,206.46.232.11,3.004,2,124,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Botnet,yes
669353,669353,18:12:06.950,147.32.84.165,147.32.80.9,0.000,1,69,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Botnet,yes
1057434,1057434,18:50:20.638,98.247.116.229,147.32.84.229,0.000,1,60,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Background,no


In [108]:
# drop un-nessary columns
o1=orig.drop(columns=['Label','Flow_Start','Source_IP','Destination_IP','Unnamed: 0'],axis=1)
mymap1 = {'yes':1,'no':0}
df_cluster = o1.applymap(lambda s: mymap1.get(s) if s in mymap1 else s)
df_cluster

,Duration,Packets,Bytes,A_,CON,ECO,ECR,FAC_,FAU_,FA_,...,832,843,877,88,888,916,98,993,995,yes/no
451907,0.000,1,60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
799441,0.000,1,71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
400755,0.000,1,74,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
858709,0.000,1,73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
533522,3.004,2,124,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
925335,0.147,3,422,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
787396,3.004,2,124,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
669353,0.000,1,69,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1057434,0.000,1,60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [109]:
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
X=df_cluster.drop(columns=['yes/no'],axis=1)
data_scaled = normalize(X)
data_scaled = pd.DataFrame(data_scaled, columns=X.columns)
data_scaled.head()

,Duration,Packets,Bytes,A_,CON,ECO,ECR,FAC_,FAU_,FA_,...,83,832,843,877,88,888,916,98,993,995
0,0.000000,0.016660,0.999584,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.014076,0.999405,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.013506,0.999453,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.013695,0.999719,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.024212,0.016120,0.999414,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<font color='blue'><font size=3>The x-axis contains the samples and y-axis represents the distance between these samples. The vertical line with maximum distance is the blue line and hence we can decide a threshold of 12 and cut the dendrogram.

In [110]:
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering
# Create a AgglomerativeClustering model with 2 clusters: model
model = AgglomerativeClustering()
# Use fit_predict to fit model and obtain cluster labels: labels
labels = model.fit_predict(data_scaled)

In [111]:
model.labels_

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [112]:
# Create a DataFrame with labels and class as columns: df
df = pd.DataFrame({'labels': labels, 'class': df_cluster['yes/no']})
# Create crosstab: ct
ct = pd.crosstab(df['labels'], df['class'])
# display ct
ct

class,0,1
labels,,
0,4411,2931
1,0,1362


In [133]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
X=df_cluster.drop(columns=['yes/no'],axis=1)
data_scaled = normalize(X)
y = df_cluster['yes/no']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3,random_state=42)
scaler_minmax = MinMaxScaler()
scaler_minmax.fit(X_train)
scaler_minmax.fit(X_test)
kmeans = KMeans(n_clusters = 2).fit(df_cluster)
labels=kmeans.fit_predict(df_cluster)
kmeans =pd.DataFrame(kmeans.labels_)

kmeans.columns = ["KMeans"]
kmeans = kmeans['KMeans']
df2=df_cluster.reset_index().drop('index', axis=1)
for index, row in df2.iterrows():
    df2.at[index, 'KMeans'] = kmeans[index]
key = df2[['yes/no', 'KMeans']]
cm = pd.DataFrame({'labels':labels, 'class':df2['yes/no']})
pd.crosstab(cm['labels'], cm['class'])

class,0,1
labels,,
0,4407,4293
1,4,0


In [134]:
from sklearn.cluster import DBSCAN 
clustering = DBSCAN(eps=3, min_samples=2).fit(df)
dbs = DBSCAN(eps=2, min_samples=3500).fit(df)
kmeans =pd.DataFrame(dbs.labels_)
kmeans.columns = ["DBSCAN"]
kmeans = kmeans['DBSCAN']
df2=df_cluster.reset_index().drop('index', axis=1)
for index, row in df2.iterrows():
    df2.at[index, 'DBSCAN'] = kmeans[index]
key = df2[['yes/no', 'DBSCAN']]
print(df2.groupby(["DBSCAN", "yes/no"]).size())

DBSCAN  yes/no
0.0     0         4411
        1         4293
dtype: int64


In [135]:
from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier(n_estimators=10).fit(X_train, y_train)
print(clf.score(X_test, y_test))
    #default estimator isn't that bad

1.0


## Naive Bayes Classifiers (taking out source city and country with balancing data)

In [113]:
# read csv dataset1 without_sourcecity down-sampling
dsampled_no_city = pd.read_csv(r'dataset1_without_sourcecity.csv')
dsampled_no_city

,Unnamed: 0,Flow_Start,Source_IP,Destination_IP,Duration,Packets,Bytes,Label,A_,CON,...,832,843,877,88,888,916,98,993,995,yes/no
0,4117,17:13:56.723,64.12.175.136,147.32.84.165,0.000,1,263,Botnet,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,yes
1,6269,17:14:06.078,212.117.171.138,147.32.84.165,0.000,1,66,Botnet,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,yes
2,8919,17:14:17.100,94.63.149.150,147.32.84.165,2.712,4,256,Botnet,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,yes
3,10127,17:14:22.915,94.63.149.150,147.32.84.165,0.000,1,64,Botnet,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,yes
4,11546,17:14:29.123,94.63.149.150,147.32.84.165,0.000,1,64,Botnet,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43515,937125,18:37:34.662,213.110.89.63,147.32.84.229,0.000,1,66,Background,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,no
43516,58713,17:18:02.785,147.32.84.229,201.250.104.62,0.000,3,2969,Background,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,no
43517,87402,17:20:15.670,147.32.84.229,107.3.105.226,0.000,1,74,Background,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,no
43518,877661,18:31:51.521,77.233.5.72,147.32.84.229,0.000,1,80,Background,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,no


In [114]:
dsampled_no_city=dsampled_no_city.drop(columns=['Flow_Start', 'Source_IP', 'Destination_IP','Label','Unnamed: 0'],axis=1)
dsampled_no_city

,Duration,Packets,Bytes,A_,CON,ECO,ECR,FAC_,FAU_,FA_,...,832,843,877,88,888,916,98,993,995,yes/no
0,0.000,1,263,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,yes
1,0.000,1,66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,yes
2,2.712,4,256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,yes
3,0.000,1,64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,yes
4,0.000,1,64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43515,0.000,1,66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,no
43516,0.000,3,2969,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,no
43517,0.000,1,74,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,no
43518,0.000,1,80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,no


In [115]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB

from sklearn import metrics
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dsampled_no_city.drop('yes/no',axis=1), dsampled_no_city['yes/no'], test_size=0.2, random_state=101)

In [116]:
#Create a Gaussian Classifier
model = GaussianNB()

# Train the model using the training sets
model.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [117]:
 model.score(X_test, y_test)

0.521484375

In [118]:
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix
y_pred = model.predict(X_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

          no       0.68      0.04      0.07      4245
         yes       0.52      0.98      0.68      4459

   micro avg       0.52      0.52      0.52      8704
   macro avg       0.60      0.51      0.37      8704
weighted avg       0.60      0.52      0.38      8704

[[ 149 4096]
 [  69 4390]]


In [140]:
clf = BaggingClassifier(n_estimators=10).fit(X_train, y_train)
print(clf.score(X_test, y_test))## whithout city and country
    #default estimator isn't that bad

0.9209558823529411


## GINI Decision Tree Classifier(taking out source city and country with balancing data)

In [119]:
dtree = DecisionTreeClassifier(criterion='gini')
dtree.fit(X_train,y_train)
predictions = dtree.predict(X_test)
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,y_pred))

[[3870  375]
 [ 296 4163]]
              precision    recall  f1-score   support

          no       0.68      0.04      0.07      4245
         yes       0.52      0.98      0.68      4459

   micro avg       0.52      0.52      0.52      8704
   macro avg       0.60      0.51      0.37      8704
weighted avg       0.60      0.52      0.38      8704



In [120]:
dtree.score(X_test, y_test)

0.9229090073529411

## Hierarchical clustering (taking out source city and country with balancing data)

In [121]:
# only take 20% of data
mymap1 = {'yes':1,'no':0}
X_cluster = dsampled_no_city.applymap(lambda s: mymap1.get(s) if s in mymap1 else s)
X_cluster1=X_cluster.sample(frac = 0.2) 
X_cluster2=X_cluster1.drop('yes/no',axis=1)


In [122]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_cluster2)
X_cluster_scaled= scaler.transform(X_cluster2)
X_cluster_scaled

array([[-0.53081428, -0.08923248, -0.08794824, ...,  0.        ,
        -0.02144225, -0.01516022],
       [ 2.05133567,  0.30253524,  0.01804477, ...,  0.        ,
        -0.02144225, -0.01516022],
       [-0.53081428, -0.08923248, -0.08513052, ...,  0.        ,
        -0.02144225, -0.01516022],
       ...,
       [-0.51308969, -0.064747  , -0.08597584, ...,  0.        ,
        -0.02144225, -0.01516022],
       [-0.48489149, -0.064747  , -0.08217192, ...,  0.        ,
        -0.02144225, -0.01516022],
       [-0.53081428, -0.08923248, -0.08808912, ...,  0.        ,
        -0.02144225, -0.01516022]])

In [123]:
# Create a AgglomerativeClustering model with 2 clusters: model
model = AgglomerativeClustering()
# Use fit_predict to fit model and obtain cluster labels: labels
labels = model.fit_predict(X_cluster_scaled)

In [124]:
model.labels_

array([1, 0, 1, ..., 0, 0, 1], dtype=int64)

In [125]:
# Create a DataFrame with labels and class as columns: df
df = pd.DataFrame({'labels': labels, 'class': X_cluster1['yes/no']})
# Create crosstab: ct
ct = pd.crosstab(df['labels'], df['class'])
# display ct
ct

class,0,1
labels,,
0,1657,2433
1,2670,1944


In [138]:
kmeans = KMeans(n_clusters = 2).fit(X_cluster_scaled)
labels=kmeans.fit_predict(X_cluster_scaled)
kmeans =pd.DataFrame(kmeans.labels_)


In [139]:
# Create a DataFrame with labels and class as columns: df
df2 = pd.DataFrame({'labels': labels, 'class': X_cluster1['yes/no']})
# Create crosstab: ct
ct = pd.crosstab(df2['labels'], df2['class'])
# display ct
ct

class,0,1
labels,,
0,2708,1958
1,1619,2419


In [143]:
from sklearn.cluster import DBSCAN 
clustering = DBSCAN(eps=3, min_samples=2).fit(X_cluster1)
dbs = DBSCAN(eps=2, min_samples=3500).fit(X_cluster1)
kmeans =pd.DataFrame(dbs.labels_)
kmeans.columns = ["DBSCAN"]
kmeans = kmeans['DBSCAN']
df2=X_cluster1.reset_index().drop('index', axis=1)
for index, row in df2.iterrows():
    df2.at[index, 'DBSCAN'] = kmeans[index]
key = df2[['yes/no', 'DBSCAN']]
print(df2.groupby(["DBSCAN", "yes/no"]).size())

DBSCAN  yes/no
-1.0    0         4327
        1         4377
dtype: int64
